# Modeling Tables for ML

Dieses Notebook baut **modellfertige Tabellen** aus den Spotify-Rohdaten.

**Erzeugt werden:**
- `track_df` – 1 Zeile = 1 Track
- `album_df` – 1 Zeile = 1 Album
- `artist_df` – 1 Zeile = 1 Artist

**Ziel:**
Saubere Features, robuste NA-Behandlung und direkte Nutzbarkeit in ML-Pipelines.

In [1]:
import sys, importlib

# reload in der richtigen Reihenfolge (dependencies zuerst, dann consumer)
for m in [
    "utils.data.parsing",
    "utils.features.numeric_transform",
    "utils.data.catalog",
    "utils.datasets_modeling.track_dataset",
    "utils.datasets_modeling.album_dataset",
"utils.features.engineering"
]:
    if m in sys.modules:
        importlib.reload(sys.modules[m])

## Imports und Setup

In [2]:
from utils.config.settings import (
    RANDOM_SEED,
    ALLOW_LEAKY_FEATURES,
    MAIN_ALBUM_STRATEGY,
)

from utils.data.catalog import TableCatalog, TableCatalogConfig
from utils.datasets_modeling.track_dataset import TrackDatasetBuilder, TrackDatasetConfig
from utils.datasets_modeling.album_dataset import AlbumDatasetBuilder, AlbumDatasetConfig
from utils.datasets_modeling.artist_dataset import ArtistDatasetBuilder, ArtistDatasetConfig
from utils.features.engineering import TrackEngineer, TrackEngineerConfig,AlbumEngineer, AlbumEngineerConfig,ArtistEngineer,ArtistEngineerConfig


from utils.core.paths import (
    load_sample_name,
    make_paths,
    ensure_dirs,
build_run_meta)

SAMPLE_NAME = load_sample_name()
PATHS = make_paths(SAMPLE_NAME)
ensure_dirs(PATHS)

RUN_META = build_run_meta(
    PATHS,
    random_seed=RANDOM_SEED,
    allow_leaky_features=ALLOW_LEAKY_FEATURES,
    main_album_strategy=MAIN_ALBUM_STRATEGY,
)



## Daten laden und vorbereiten

In [3]:
TABLES = [
    "tracks","audio_features","albums","artists","genres",
    "r_albums_tracks","r_track_artist","r_artist_genre","r_albums_artists"
]

catalog = TableCatalog(TableCatalogConfig(PATHS.clean_parquet_dir, TABLES))
data = catalog.load(strict=False)

## Track-Level Dataset (1 Zeile = 1 Track)

**Ziel:**
Aufbau einer **denormalisierten, modellierungsfertigen Feature-Tabelle (`track_df`)**,
in der **jede Zeile genau einen Track** repräsentiert.

Das Dataset dient als **zentrale Input-Matrix für Track-basierte ML-Modelle**
(z. B. Popularity-Regression, Hit-Prediction, Genre-Klassifikation).

---

### Pipeline & Designentscheidungen

### 1️⃣ Tracks + Audio-Features
Die Track-Tabelle wird per **Left Join** mit den numerischen Audio-Features
(z. B. `energy`, `danceability`, `loudness`, `tempo`) angereichert.

- Fehlende Audio-Features bleiben explizit `NaN`
- Keine Imputation an dieser Stelle → Trennung von **Datenaufbereitung** und **Modelllogik**

**Motivation:**
Audio-Features sind zentrale Prädiktoren, aber nicht für alle Tracks verfügbar
(z. B. lokale Uploads, ältere Releases).

---

### 2️⃣ Track → Album (M:N → 1)
Ein Track kann mehreren Alben zugeordnet sein (z. B. Compilation, Re-Release).

→ Es wird **deterministisch ein Main Album** pro Track gewählt
(z. B. *frühestes Release* oder *primäres Studioalbum*).

**Motivation:**
- Eindeutige Album-Zuordnung ist notwendig für ML-Modelle
- Vermeidet Daten-Leakage durch Mehrfachzählungen

---

### 3️⃣ Album-Metadaten & Zeitfeatures
Album-Informationen werden auf Track-Ebene gemerged, inklusive:

- `release_year`
- `release_month`
- `release_decade`

**Motivation:**
Zeitliche Effekte (Ära, Streaming-Phase, Marktbedingungen) haben starken Einfluss
auf Track-Performance.

---

### 4️⃣ Track → Artists (M:N → Aggregation)
Ein Track kann mehrere Artists haben (Features, Kollaborationen).

Gespeichert werden:
- `artist_ids` (Liste)
- Aggregierte Artist-Statistiken:
  - Anzahl Artists (`n_artists`)
  - Mittelwert / Maximum von Popularity
  - Mittelwert / Maximum von Followers

**Motivation:**
- Kollaborationen verhalten sich anders als Solo-Tracks
- Star-Power einzelner Artists ist oft entscheidend

---

### 5️⃣ Genres über Artists
Genres sind im Schema **an Artists gebunden**, nicht direkt an Tracks.

→ Für jeden Track:
- Union aller Artist-Genres
- Speicherung als Liste: `track_genres`

**Motivation:**
- Genre ist ein *kontextuelles* Attribut
- Mehrere Genres pro Track sind realistisch und informativ

---

### 6️⃣ Feature Engineering (Track-Level)
Zusätzliche robuste Features:

- **Text & Meta**
  - `has_preview`
  - `name_len`, `name_words`
- **Numerische Transformationen**
  - Log-Skalen für Dauer & Follower-Counts
- **Qualitäts-Flags**
  - `has_audio_features`
  - `is_collab`

**Motivation:**
Stabilisierung von Skalen, Reduktion von Ausreißern,
bessere Lernbarkeit für ML-Modelle.

---

### 📤 Output
**`track_df`**
Eine konsistente, ML-fertige Feature-Tabelle auf **Track-Ebene**
(1 Zeile = 1 Track).


In [4]:
track_df =  TrackDatasetBuilder(config=TrackDatasetConfig(main_album_strategy="earliest_release")).build(data)
track_df = TrackEngineer(TrackEngineerConfig()).transform(track_df)
track_df.head(3)

,track_id,disc_number,duration,explicit,audio_feature_id,name,track_number,popularity,has_preview,is_long_track,...,mood_calm_happy,mood_sad_calm,n_genres,has_genre,main_genre_id,is_modern_release,is_old_release,is_release_year_missing,is_tracknum_missing,is_disc_missing
0,0JoQKXwyqwEe90f7XlT5UZ,1,145400,False,0JoQKXwyqwEe90f7XlT5UZ,Let Go,6,9,1,0,...,0,0,1,1,finnish punk,0,0,0,0,0
1,0jU3GXkrmuXkIWSubTQG6q,1,514373,False,0jU3GXkrmuXkIWSubTQG6q,"We Came from the Dust, but That Isn't Where We...",7,0,0,0,...,0,1,0,0,<NA>,1,0,0,0,0
2,0jxcw9bsXmHbPUSZqscNai,1,259000,False,0jxcw9bsXmHbPUSZqscNai,Amie - Remastered January 1992,1,38,1,0,...,0,1,5,1,classic rock,0,0,0,0,0


In [5]:
track_df.columns

Index(['track_id', 'disc_number', 'duration', 'explicit', 'audio_feature_id',
       'name', 'track_number', 'popularity', 'has_preview', 'is_long_track',
       'is_tracknum_extreme', 'is_multidisc', 'is_disc_extreme',
       'acousticness', 'analysis_url', 'danceability', 'duration_af', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence',
       'is_time_signature_rare', 'is_tempo_extreme', 'is_loudness_very_low',
       'is_af_long', 'is_high_speech', 'is_instrumental',
       'track_release_date_parsed', 'album_id', 'release_date_parsed',
       'album_popularity', 'album_release_date_raw', 'release_year',
       'release_month', 'release_decade', 'artist_ids', 'n_artists',
       'artist_popularity_mean', 'artist_popularity_max',
       'artist_followers_mean', 'artist_followers_max', 'track_genres',
       'is_explicit', 'has_audio_features', 'is_collab', 'name_len',
       'name_words', 'log_durat

## Album-Level Dataset (1 Zeile = 1 Album)

**Ziel:**
Aufbau einer **aggregierten, ML-tauglichen Feature-Matrix (`album_df`)**,
in der **jede Zeile ein Album** repräsentiert.

Der Fokus liegt auf **strukturellen, aggregierten und kontextuellen Features**,
die Album-Charakteristika erfassen – unabhängig von einzelnen Tracks.

---

### Pipeline & Designentscheidungen

### 1️⃣ Album-Metadaten & Zeitfeatures
Robustes Parsing des Release-Datums mit Ableitung von:

- `release_year`
- `release_month`
- `release_decade`

**Motivation:**
Alben unterliegen starken **Era-Effekten** (z. B. Vinyl-Ära vs. Streaming-Ära).

---

### 2️⃣ Album-Größe (Strukturmerkmal)
Berechnung der **Anzahl Tracks pro Album** (`n_tracks`)
über die Album–Track-Relation.

**Motivation:**
- Singles, EPs, LPs unterscheiden sich systematisch
- Albumgröße ist ein starker struktureller Prädiktor

---

### 3️⃣ Album-Audio-Profil (Track-Aggregation)
Aggregierte Audio-Signatur pro Album:

- Mittelwerte von:
  - `energy`, `danceability`
  - `loudness`, `tempo`
  - `speechiness`, `valence`, …

**Motivation:**
Ein Album besitzt einen **akustischen Gesamteindruck**,
der über einzelne Tracks hinausgeht.

---

### 4️⃣ Album-Artist-Profil
Aggregation der beteiligten Artists pro Album:

- Anzahl Artists (`n_album_artists`)
- Mittelwert / Maximum von:
  - Artist Popularity
  - Artist Followers

**Motivation:**
- Kollaborations-Alben vs. Solo-Alben
- Einfluss von „Headliner“-Artists

---

### 5️⃣ Album-Genres
Genres werden als **Union aller Artist-Genres** definiert:

- Speicherung als Liste: `album_genres`
- Ableitung:
  - Anzahl Genres
  - Hauptgenre (Heuristik)
  - Multi-Genre-Flag

**Motivation:**
Alben sind häufig **genreübergreifend**, besonders bei Compilations
oder modernen Releases.

---

### 6️⃣ Feature Engineering (Album-Level)
Zusätzliche robuste Features:

- **Skalentransformation**
  - `log_n_tracks`
- **Textfeatures**
  - `name_len`, `name_words`
- **Strukturelle Flags**
  - Single / EP / LP / Mega-Album
- **Era- & Recency-Features**
- **Audio-Mood-Signaturen**
- **Artist-Power-Indikatoren**

---

### 📤 Output
**`album_df`**
Eine aggregierte Feature-Matrix auf **Album-Ebene**
(1 Zeile = 1 Album), geeignet für Album-basierte Analysen
und ML-Modelle.


In [6]:
album_df = AlbumDatasetBuilder(AlbumDatasetConfig()).build(data)
album_df = AlbumEngineer(AlbumEngineerConfig()).transform(album_df)
album_df.head(3)

,album_id,name,album_type,release_date,popularity,release_date_parsed,is_release_year_invalid,release_year,release_month,release_decade,...,album_is_collab,album_artist_followers_mean_log1p,album_artist_followers_max_log1p,album_artist_followers_gap,album_artist_popularity_gap,album_has_headliner_artist,n_album_genres,album_has_genre,album_main_genre_id,album_is_multi_genre
0,7zzsmehid2XISbO0viFilJ,Dikke Trienne,single,1427760000000,1,2015-03-31,0,2015,3,2010,...,0,13.941222,13.941222,0.0,0.0,0,0,0,<NA>,0
1,000J6XlimZiqvogOm2eIzr,The Folk Sound of Jimmie Rodgers,album,999993600000,0,2001-09-09,0,2001,9,2000,...,0,0.000000,0.000000,0.0,0.0,0,9,1,appalachian folk,1
2,000VJzggsEiQmNpLpIUQmD,La Copla,compilation,915148800000,11,1999-01-01,0,1999,1,1990,...,0,13.941222,13.941222,0.0,0.0,0,0,0,<NA>,0


In [7]:
album_df.columns

Index(['album_id', 'name', 'album_type', 'release_date', 'popularity',
       'release_date_parsed', 'is_release_year_invalid', 'release_year',
       'release_month', 'release_decade', 'n_tracks',
       'album_mean_acousticness', 'album_mean_danceability',
       'album_mean_energy', 'album_mean_instrumentalness',
       'album_mean_liveness', 'album_mean_speechiness', 'album_mean_valence',
       'album_mean_loudness', 'album_mean_tempo', 'n_album_artists',
       'album_artist_popularity_mean', 'album_artist_popularity_max',
       'album_artist_followers_mean', 'album_artist_followers_max',
       'album_genres', 'log_n_tracks', 'name_len', 'name_words',
       'album_name_len', 'album_name_words', 'is_deluxe', 'is_remaster',
       'is_live_album', 'is_compilation', 'is_single', 'is_ep', 'is_lp',
       'is_mega_album', 'is_n_tracks_missing', 'is_modern_release',
       'is_old_release', 'is_release_year_missing', 'release_year_norm',
       'album_mood_happy_energetic', 'album_m

## Artist-Level Dataset (1 Zeile = 1 Artist)

**Ziel:** Aufbau einer ML-fertigen Tabelle (`artist_df`) auf **Artist-Ebene**.
Der Fokus liegt auf **Clustering, Similarity Search und Community Detection**, optional auch als Input für supervised Modelle.

Jede Zeile repräsentiert **einen Artist** mit einem stabilen numerischen **Style- und Popularitätsprofil**.

---

### Pipeline-Überblick

#### 1. Artist-Stammdaten
Start mit der Tabelle `artists`.
Der Primärschlüssel wird konsistent als `artist_id` geführt, um stabile Joins über alle Beziehungstabellen zu gewährleisten.

---

#### 2. Artist-Style-Profil (Aggregation über Tracks)
Über die M:N-Beziehung **Artist ↔ Track** werden Track-Features auf Artist-Ebene aggregiert:

- **Produktivität**
  - Anzahl eindeutiger Tracks (`n_tracks`)

- **Track-basierte Popularität**
  - Ø Track-Popularity (`track_pop_mean`, falls vorhanden)
  - Explicit-Rate (`explicit_rate`, Anteil expliziter Tracks)

- **Audio-Signatur (Artist-Sound-Vektor)**
  - Mittelwerte zentraler Audio-Features
    *(z. B. Energy, Danceability, Valence, Tempo, Loudness, …)*

Diese Aggregation liefert einen **kompakten, stabilen Style-Vektor pro Artist**.

---

#### 3. Artist-Genres
Merge der Genre-Informationen pro Artist:

- `artist_genres`: Liste von Genre-IDs
- Genres bleiben bewusst als **Listen**, um spätere Multi-Hot-Encodings (Top-K Genres) zu ermöglichen.

---

#### 4. Feature Engineering
Anwendung von regelbasiertem Feature Engineering zur Verbesserung der Modellierbarkeit:

- **Numerische Stabilisierung**
  - Typ-Sicherheit für Zähl- und Audio-Features

- **Heavy-Tail-Transformationen**
  - `followers_log1p`
  - `n_tracks_log1p`
  - `followers_per_track` (+ Log-Variante)

- **Popularitäts-Signale**
  - Relative Flags (z. B. Headliner-Artists)
  - Gap zwischen Artist- und Track-Popularity

- **Style- & Mood-Features**
  - Mood-Quadranten (Energy × Valence)
  - Dance-Energy-Interaktionen
  - Tempo-, Speech- und Instrumental-Flags (relativ zur Dataset-Verteilung)

- **Genre-Features**
  - Anzahl Genres
  - Hauptgenre-Heuristik
  - Multi-Genre-Flag (stilistische Breite)

---

### Output

**`artist_df`** – eine konsistente Feature-Matrix auf Artist-Ebene mit:

- numerischem Style-Vektor
- Popularitäts- und Produktivitäts-Signalen
- Genre-Informationen

Geeignet für:
- Clustering & Ähnlichkeitsanalysen
- Artist-Embeddings
- Graph- & Community-Modelle
- optionale supervised Learning Tasks

In [8]:
artist_df = ArtistDatasetBuilder(ArtistDatasetConfig()).build(data, track_df=track_df)
artist_df = ArtistEngineer(ArtistEngineerConfig()).transform(artist_df, verbose=True)
artist_df.head(3)

ArtistEngineer created 23 features
   -> ['acoustic_minus_energy', 'artist_mood_angry_energetic', 'artist_mood_calm_happy', 'artist_mood_happy_energetic', 'artist_mood_sad_calm', 'artist_vs_track_pop_gap', 'dance_x_energy', 'explicit_rate_filled', 'followers_per_track', 'followers_per_track_log1p', 'has_genre', 'is_fast_tempo_artist', 'is_headliner_popularity', 'is_high_speech_artist', 'is_instrumental_heavy_artist', 'is_mostly_explicit', 'is_multi_genre', 'is_never_explicit', 'is_quiet_artist', 'is_slow_tempo_artist', 'main_genre_id', 'n_artist_genres', 'n_tracks_log1p'] 


,artist_id,name,popularity,followers,is_followers_extreme,followers_log1p,artist_genres,n_tracks,track_pop_mean,explicit_rate,...,is_slow_tempo_artist,is_fast_tempo_artist,acoustic_minus_energy,is_high_speech_artist,is_instrumental_heavy_artist,is_quiet_artist,n_artist_genres,has_genre,main_genre_id,is_multi_genre
0,7zzsdcNemyhcNk2wpNsXZt,Sinéad Lohan,31,3377,0,8.125039,"[irish singer-songwriter, lilith]",1,8.0,0.0,...,0,1,0.698000,0,0,1,2,1,irish singer-songwriter,0
1,0008buRx5YKdlfkuO6zPU7,Alan Bremner,0,0,0,0.000000,[],1,0.0,0.0,...,0,0,-0.729956,0,1,0,0,0,<NA>,0
2,000Nolc0A0ZuWkFzNYVYdU,Peter Clark,0,0,0,0.000000,[],1,0.0,0.0,...,0,0,0.947200,0,0,1,0,0,<NA>,0


In [9]:
artist_df.columns

Index(['artist_id', 'name', 'popularity', 'followers', 'is_followers_extreme',
       'followers_log1p', 'artist_genres', 'n_tracks', 'track_pop_mean',
       'explicit_rate', 'mean_acousticness', 'mean_danceability',
       'mean_energy', 'mean_instrumentalness', 'mean_liveness',
       'mean_speechiness', 'mean_valence', 'mean_loudness', 'mean_tempo',
       'log_followers', 'log_n_tracks', 'n_tracks_log1p',
       'followers_per_track', 'followers_per_track_log1p',
       'is_headliner_popularity', 'artist_vs_track_pop_gap',
       'is_mostly_explicit', 'is_never_explicit', 'explicit_rate_filled',
       'artist_mood_happy_energetic', 'artist_mood_angry_energetic',
       'artist_mood_calm_happy', 'artist_mood_sad_calm', 'dance_x_energy',
       'is_slow_tempo_artist', 'is_fast_tempo_artist', 'acoustic_minus_energy',
       'is_high_speech_artist', 'is_instrumental_heavy_artist',
       'is_quiet_artist', 'n_artist_genres', 'has_genre', 'main_genre_id',
       'is_multi_genre'],
   

## Speichere Modelling Tables

In [10]:
track_out = PATHS.modeling_dir / "track_dataset.parquet"
album_out = PATHS.modeling_dir / "album_dataset.parquet"
artist_out = PATHS.modeling_dir / "artist_dataset.parquet"

track_df.to_parquet(track_out, index=False)
album_df.to_parquet(album_out, index=False)
artist_df.to_parquet(artist_out, index=False)

print(" Saved modeling datasets_modeling:")
print(" -", track_out)
print(" -", album_out)
print(" -", artist_out)

 Saved modeling datasets_modeling:
 - C:\GitHub\uni-project-metrics-and-data\data\processed\modeling\slice_000\track_dataset.parquet
 - C:\GitHub\uni-project-metrics-and-data\data\processed\modeling\slice_000\album_dataset.parquet
 - C:\GitHub\uni-project-metrics-and-data\data\processed\modeling\slice_000\artist_dataset.parquet
